In [19]:
import numpy as np
import scipy as sp
import astropy as ap
import glob
import pandas as pd
import matplotlib.pyplot as plt
from scipy.integrate import quad
import scipy.constants as const
from astropy import constants as ap_const
import scipy.ndimage 
import PIL
from mpdaf.obj import Image,WCS
import sptpol_software as sps
from astropy.io import fits
from astropy.cosmology import Planck13
from astropy import units as u
import scipy.spatial.distance as dist
import os 
# os.chdir("/home/mitchell/Documents/masters/masters/data")
cwd = os.getcwd()
print(cwd)
print(Planck13)
cosmo= Planck13
import scipy.spatial.distance as dist
from scipy import sparse
from scipy.spatial import cKDTree as KDTree

/home/mitchell/Documents/masters/masters/data
FlatLambdaCDM(name="Planck13", H0=67.8 km / (Mpc s), Om0=0.307, Tcmb0=2.725 K, Neff=3.05, m_nu=[ 0.    0.    0.06] eV, Ob0=0.0483)


In [16]:
def getPairs(data_frame, max_sep=20, query_type = 1,results_loc='PAIRS_sparse_dist.npz', save_frame=False, output='DES_REDMAGIC_Manipulated.csv'):
    '''
    Takes a data frame with RA, DEC, and COMOVING distances, and pairs
    up those vectors with the closest vector under a given maximum separation
    and store the results in an array where:
    results[0] = First Item
    results[1] = Second Item
    results[2] = Distance between First Item and Second Item
    '''
    if 'RA' not in data_frame.columns:
        return("Error: RA not in Data Frame")
    if 'DEC' not in data_frame.columns:
        return("Error: DEC not in Data Frame")
#     if 'COMOVING' not in data_frame.columns:
#         return("Error: COMOVINGb not in Data Frame")

#     vec_distances = []
#     for index in range(len(data_frame)):
#         vec_distances.append(get_vec_distances(
#             data_frame['RA'][index], data_frame['DEC'][index]))
# #         , data_frame['COMOVING'][index]

#     data_frame['x_vec'] = pd.Series(np.transpose(vec_distances)[0])
#     data_frame['y_vec'] = pd.Series(np.transpose(vec_distances)[1])
#     data_frame['z_vec'] = pd.Series(np.transpose(vec_distances)[2])

    # vec = [data_frame['x_vec'][0], data_frame['y_vec']
    #    [0], data_frame['z_vec'][0]]
    vec_frame = np.vstack(
        (data_frame['RA'], data_frame['DEC']))
#     , data_frame['z_vec']
    vec_frame = vec_frame.T

    frame_tree = KDTree(vec_frame)
    if query_type == 1:
        dist_matr = frame_tree.sparse_distance_matrix(
            frame_tree, max_distance=max_sep, p=2.0, output_type='ndarray')
    elif query_type == 2:
            dist_matr = frame_tree.query_pairs(r=max_sep, p=2.0, output_type='ndarray')
    dist_u = dist_matr[dist_matr['i'] < dist_matr['j']]
    result = sparse.coo_matrix(
        (dist_u['v'], (dist_u['i'], dist_u['j'])), (len(dist_u), len(dist_u)))

    if save_frame == True:
        data_frame.to_csv(output)

    sp.sparse.save_npz(results_loc, result)

    pairs_matrix = np.vstack((dist_u['i'], dist_u['j'], dist_u['v']))

    return(pairs_matrix)

In [8]:
from astropy.table import Table
with fits.open("DES_Y1A1_3x2pt_redMaGiC_zerr_CATALOG.fits") as data:
    df = pd.DataFrame(data[1].data)

In [18]:
pairs = getPairs(df,3)

MemoryError: 